In [2]:
import numpy as np
#from lhereader import LHEReader

from LHCO_reader import LHCO_reader

import math
import os

# Plotting library
import matplotlib.pyplot as plt

### extract info for LQ analysis

## SIGNAL LQ down

#### LQ down 800GeV

In [4]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQdown-800GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQdown-800GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

/home/andres/anaconda3/envs/py2/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:359: UserWarning: Couldn't read total number of events from LHCO
  warnings.warn("Couldn't read total number of events from LHCO")


('run: ', 1, '   # events ok so far: ', 2379)
('run: ', 2, '   # events ok so far: ', 4720)
('run: ', 3, '   # events ok so far: ', 7024)
('run: ', 4, '   # events ok so far: ', 9335)
('run: ', 5, '   # events ok so far: ', 11583)
('run: ', 6, '   # events ok so far: ', 13890)
('run: ', 7, '   # events ok so far: ', 16205)
('run: ', 8, '   # events ok so far: ', 18546)
('run: ', 9, '   # events ok so far: ', 20815)
('run: ', 10, '   # events ok so far: ', 23131)
('run: ', 11, '   # events ok so far: ', 25369)
('run: ', 12, '   # events ok so far: ', 27662)
('run: ', 13, '   # events ok so far: ', 29962)
('run: ', 14, '   # events ok so far: ', 32312)
('run: ', 15, '   # events ok so far: ', 34565)
('run: ', 16, '   # events ok so far: ', 36830)
('run: ', 17, '   # events ok so far: ', 39122)
('run: ', 18, '   # events ok so far: ', 41445)
('run: ', 19, '   # events ok so far: ', 43819)
('run: ', 20, '   # events ok so far: ', 46246)
('run: ', 21, '   # events ok so far: ', 48592)
('run

('run: ', 168, '   # events ok so far: ', 389147)
('run: ', 169, '   # events ok so far: ', 391410)
('run: ', 170, '   # events ok so far: ', 393770)
('run: ', 171, '   # events ok so far: ', 396106)
('run: ', 172, '   # events ok so far: ', 398415)
('run: ', 173, '   # events ok so far: ', 400699)
('run: ', 174, '   # events ok so far: ', 402983)
('run: ', 175, '   # events ok so far: ', 405324)
('run: ', 176, '   # events ok so far: ', 407520)
('run: ', 177, '   # events ok so far: ', 409821)
('run: ', 178, '   # events ok so far: ', 412153)
('run: ', 179, '   # events ok so far: ', 414488)
('run: ', 180, '   # events ok so far: ', 416816)
('run: ', 181, '   # events ok so far: ', 419130)
('run: ', 182, '   # events ok so far: ', 421351)
('run: ', 183, '   # events ok so far: ', 423605)
('run: ', 184, '   # events ok so far: ', 425877)
('run: ', 185, '   # events ok so far: ', 428140)
('run: ', 186, '   # events ok so far: ', 430355)
('run: ', 187, '   # events ok so far: ', 432664)


In [ ]:
# total LQ down 800GeV     462728

In [13]:
2379*200

475800

#### LQ down 1000GeV

In [7]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQdown-1000GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQdown-1000GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2445)
('run: ', 2, '   # events ok so far: ', 4953)
('run: ', 3, '   # events ok so far: ', 7491)
('run: ', 4, '   # events ok so far: ', 10058)
('run: ', 5, '   # events ok so far: ', 12479)
('run: ', 6, '   # events ok so far: ', 14994)
('run: ', 7, '   # events ok so far: ', 17482)
('run: ', 8, '   # events ok so far: ', 20020)
('run: ', 9, '   # events ok so far: ', 22520)
('run: ', 10, '   # events ok so far: ', 24962)
('run: ', 11, '   # events ok so far: ', 27470)
('run: ', 12, '   # events ok so far: ', 29915)
('run: ', 13, '   # events ok so far: ', 32518)
('run: ', 14, '   # events ok so far: ', 35036)
('run: ', 15, '   # events ok so far: ', 37502)
('run: ', 16, '   # events ok so far: ', 39976)
('run: ', 17, '   # events ok so far: ', 42423)
('run: ', 18, '   # events ok so far: ', 44877)
('run: ', 19, '   # events ok so far: ', 47463)
('run: ', 20, '   # events ok so far: ', 49924)
('run: ', 21, '   # events ok so far: ', 52420)
('ru

('run: ', 168, '   # events ok so far: ', 419414)
('run: ', 169, '   # events ok so far: ', 421869)
('run: ', 170, '   # events ok so far: ', 424327)
('run: ', 171, '   # events ok so far: ', 426851)
('run: ', 172, '   # events ok so far: ', 429348)
('run: ', 173, '   # events ok so far: ', 431823)
('run: ', 174, '   # events ok so far: ', 434293)
('run: ', 175, '   # events ok so far: ', 436705)
('run: ', 176, '   # events ok so far: ', 439144)
('run: ', 177, '   # events ok so far: ', 441686)
('run: ', 178, '   # events ok so far: ', 444247)
('run: ', 179, '   # events ok so far: ', 446784)
('run: ', 180, '   # events ok so far: ', 449242)
('run: ', 181, '   # events ok so far: ', 451787)
('run: ', 182, '   # events ok so far: ', 454187)
('run: ', 183, '   # events ok so far: ', 456646)
('run: ', 184, '   # events ok so far: ', 459117)
('run: ', 185, '   # events ok so far: ', 461680)
('run: ', 186, '   # events ok so far: ', 464173)
('run: ', 187, '   # events ok so far: ', 466696)


In [ ]:
# total LQ down 1000GeV     499346

In [14]:
2445*200

489000

#### LQ down 1200GeV

In [8]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQdown-1200GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQdown-1200GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2443)
('run: ', 2, '   # events ok so far: ', 4923)
('run: ', 3, '   # events ok so far: ', 7368)
('run: ', 4, '   # events ok so far: ', 9813)
('run: ', 5, '   # events ok so far: ', 12320)
('run: ', 6, '   # events ok so far: ', 14871)
('run: ', 7, '   # events ok so far: ', 17332)
('run: ', 8, '   # events ok so far: ', 19797)
('run: ', 9, '   # events ok so far: ', 22340)
('run: ', 10, '   # events ok so far: ', 24803)
('run: ', 11, '   # events ok so far: ', 27252)
('run: ', 12, '   # events ok so far: ', 29731)
('run: ', 13, '   # events ok so far: ', 32238)
('run: ', 14, '   # events ok so far: ', 34659)
('run: ', 15, '   # events ok so far: ', 37156)
('run: ', 16, '   # events ok so far: ', 39667)
('run: ', 17, '   # events ok so far: ', 42126)
('run: ', 18, '   # events ok so far: ', 44613)
('run: ', 19, '   # events ok so far: ', 47065)
('run: ', 20, '   # events ok so far: ', 49566)
('run: ', 21, '   # events ok so far: ', 52046)
('run

('run: ', 168, '   # events ok so far: ', 415193)
('run: ', 169, '   # events ok so far: ', 417751)
('run: ', 170, '   # events ok so far: ', 420116)
('run: ', 171, '   # events ok so far: ', 422633)
('run: ', 172, '   # events ok so far: ', 425123)
('run: ', 173, '   # events ok so far: ', 427586)
('run: ', 174, '   # events ok so far: ', 430114)
('run: ', 175, '   # events ok so far: ', 432619)
('run: ', 176, '   # events ok so far: ', 435121)
('run: ', 177, '   # events ok so far: ', 437593)
('run: ', 178, '   # events ok so far: ', 440040)
('run: ', 179, '   # events ok so far: ', 442496)
('run: ', 180, '   # events ok so far: ', 444940)
('run: ', 181, '   # events ok so far: ', 447452)
('run: ', 182, '   # events ok so far: ', 449767)
('run: ', 183, '   # events ok so far: ', 452239)
('run: ', 184, '   # events ok so far: ', 454617)
('run: ', 185, '   # events ok so far: ', 457109)
('run: ', 186, '   # events ok so far: ', 459473)
('run: ', 187, '   # events ok so far: ', 461886)


In [ ]:
# total LQ down 1200GeV     494133

In [15]:
2443*200

488600

#### LQ down 1400GeV

In [10]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQdown-1400GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQdown-1400GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2335)
('run: ', 2, '   # events ok so far: ', 4717)
('run: ', 3, '   # events ok so far: ', 7099)
('run: ', 4, '   # events ok so far: ', 9466)
('run: ', 5, '   # events ok so far: ', 11903)
('run: ', 6, '   # events ok so far: ', 14285)
('run: ', 7, '   # events ok so far: ', 16626)
('run: ', 8, '   # events ok so far: ', 19014)
('run: ', 9, '   # events ok so far: ', 21335)
('run: ', 10, '   # events ok so far: ', 23647)
('run: ', 11, '   # events ok so far: ', 26036)
('run: ', 12, '   # events ok so far: ', 28359)
('run: ', 13, '   # events ok so far: ', 30709)
('run: ', 14, '   # events ok so far: ', 33043)
('run: ', 15, '   # events ok so far: ', 35459)
('run: ', 16, '   # events ok so far: ', 37924)
('run: ', 17, '   # events ok so far: ', 40289)
('run: ', 18, '   # events ok so far: ', 42671)
('run: ', 19, '   # events ok so far: ', 45065)
('run: ', 20, '   # events ok so far: ', 47451)
('run: ', 21, '   # events ok so far: ', 49916)
('run

('run: ', 168, '   # events ok so far: ', 398498)
('run: ', 169, '   # events ok so far: ', 400856)
('run: ', 170, '   # events ok so far: ', 403158)
('run: ', 171, '   # events ok so far: ', 405538)
('run: ', 172, '   # events ok so far: ', 407934)
('run: ', 173, '   # events ok so far: ', 410257)
('run: ', 174, '   # events ok so far: ', 412642)
('run: ', 175, '   # events ok so far: ', 414995)
('run: ', 176, '   # events ok so far: ', 417327)
('run: ', 177, '   # events ok so far: ', 419672)
('run: ', 178, '   # events ok so far: ', 422022)
('run: ', 179, '   # events ok so far: ', 424405)
('run: ', 180, '   # events ok so far: ', 426781)
('run: ', 181, '   # events ok so far: ', 429245)
('run: ', 182, '   # events ok so far: ', 431634)
('run: ', 183, '   # events ok so far: ', 433976)
('run: ', 184, '   # events ok so far: ', 436297)
('run: ', 185, '   # events ok so far: ', 438633)
('run: ', 186, '   # events ok so far: ', 441012)
('run: ', 187, '   # events ok so far: ', 443424)


In [ ]:
# total LQ down 1400GeV     474516

In [16]:
2335*200

467000

#### LQ down 1600GeV

In [11]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQdown-1600GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQdown-1600GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2293)
('run: ', 2, '   # events ok so far: ', 4595)
('run: ', 3, '   # events ok so far: ', 6824)
('run: ', 4, '   # events ok so far: ', 9036)
('run: ', 5, '   # events ok so far: ', 11263)
('run: ', 6, '   # events ok so far: ', 13480)
('run: ', 7, '   # events ok so far: ', 15712)
('run: ', 8, '   # events ok so far: ', 17940)
('run: ', 9, '   # events ok so far: ', 20139)
('run: ', 10, '   # events ok so far: ', 22391)
('run: ', 11, '   # events ok so far: ', 24678)
('run: ', 12, '   # events ok so far: ', 26900)
('run: ', 13, '   # events ok so far: ', 29124)
('run: ', 14, '   # events ok so far: ', 31353)
('run: ', 15, '   # events ok so far: ', 33627)
('run: ', 16, '   # events ok so far: ', 35883)
('run: ', 17, '   # events ok so far: ', 38022)
('run: ', 18, '   # events ok so far: ', 40252)
('run: ', 19, '   # events ok so far: ', 42453)
('run: ', 20, '   # events ok so far: ', 44747)
('run: ', 21, '   # events ok so far: ', 46957)
('run

('run: ', 168, '   # events ok so far: ', 375833)
('run: ', 169, '   # events ok so far: ', 378046)
('run: ', 170, '   # events ok so far: ', 380294)
('run: ', 171, '   # events ok so far: ', 382512)
('run: ', 172, '   # events ok so far: ', 384798)
('run: ', 173, '   # events ok so far: ', 387048)
('run: ', 174, '   # events ok so far: ', 389311)
('run: ', 175, '   # events ok so far: ', 391522)
('run: ', 176, '   # events ok so far: ', 393792)
('run: ', 177, '   # events ok so far: ', 396026)
('run: ', 178, '   # events ok so far: ', 398204)
('run: ', 179, '   # events ok so far: ', 400451)
('run: ', 180, '   # events ok so far: ', 402615)
('run: ', 181, '   # events ok so far: ', 404880)
('run: ', 182, '   # events ok so far: ', 407128)
('run: ', 183, '   # events ok so far: ', 409425)
('run: ', 184, '   # events ok so far: ', 411706)
('run: ', 185, '   # events ok so far: ', 413934)
('run: ', 186, '   # events ok so far: ', 416200)
('run: ', 187, '   # events ok so far: ', 418407)


In [6]:
# total LQ down 1600GeV     447538

In [17]:
2293*200

458600

#### LQ down 1800GeV

In [12]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQdown-1800GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQdown-1800GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2071)
('run: ', 2, '   # events ok so far: ', 4210)
('run: ', 3, '   # events ok so far: ', 6262)
('run: ', 4, '   # events ok so far: ', 8288)
('run: ', 5, '   # events ok so far: ', 10387)
('run: ', 6, '   # events ok so far: ', 12419)
('run: ', 7, '   # events ok so far: ', 14574)
('run: ', 8, '   # events ok so far: ', 16709)
('run: ', 9, '   # events ok so far: ', 18767)
('run: ', 10, '   # events ok so far: ', 20852)
('run: ', 11, '   # events ok so far: ', 22889)
('run: ', 12, '   # events ok so far: ', 25012)
('run: ', 13, '   # events ok so far: ', 27120)
('run: ', 14, '   # events ok so far: ', 29202)
('run: ', 15, '   # events ok so far: ', 31320)
('run: ', 16, '   # events ok so far: ', 33368)
('run: ', 17, '   # events ok so far: ', 35388)
('run: ', 18, '   # events ok so far: ', 37397)
('run: ', 19, '   # events ok so far: ', 39553)
('run: ', 20, '   # events ok so far: ', 41610)
('run: ', 21, '   # events ok so far: ', 43754)
('run

('run: ', 169, '   # events ok so far: ', 353846)
('run: ', 170, '   # events ok so far: ', 355962)
('run: ', 171, '   # events ok so far: ', 358035)
('run: ', 172, '   # events ok so far: ', 360051)
('run: ', 173, '   # events ok so far: ', 362156)
('run: ', 174, '   # events ok so far: ', 364302)
('run: ', 175, '   # events ok so far: ', 366360)
('run: ', 176, '   # events ok so far: ', 368493)
('run: ', 177, '   # events ok so far: ', 370635)
('run: ', 178, '   # events ok so far: ', 372710)
('run: ', 179, '   # events ok so far: ', 374770)
('run: ', 180, '   # events ok so far: ', 376895)
('run: ', 181, '   # events ok so far: ', 378959)
('run: ', 182, '   # events ok so far: ', 381081)
('run: ', 183, '   # events ok so far: ', 383141)
('run: ', 184, '   # events ok so far: ', 385226)
('run: ', 185, '   # events ok so far: ', 387327)
('run: ', 186, '   # events ok so far: ', 389384)
('run: ', 187, '   # events ok so far: ', 391480)
('run: ', 188, '   # events ok so far: ', 393600)


In [ ]:
# total LQ down 1800GeV     418625

In [18]:
2071*200

414200